In [2]:
from IPython.display import Latex
import numpy as np
import tensorflow as tf

# 基本概念
## 梯度下降法和最小二乘法（正规方程）的对比
假设有m个训练例子，n个需要训练的特征值
### 梯度下降法
单次迭代公式：$w:= w+\alpha X(y-X^Tw)$

* 需要选择学习率$\alpha$
* 需要多次迭代
* 在n很大的时候工作的也很好

### 最小二乘法
公式：$w := (XX^T)^{-1}Xy$

* 不需要选择学习率$\alpha$
* 不需要迭代
* 需要计算$(XX^T)^{-1}$
* 当n很大时运行非常慢

$XX^T$不可逆的情况
1. 存在特征线性相关，即有多余的特征
解决方案：删除多余的特征
2. 样本数量小于特征数量
解决方案：删除部分特征，或者使用正则化方法

## 反向传播
将预测结果的偏差传递到各个参数上，根据这些参数对偏差的贡献的大小，相应地承担修改的责任
## one-hot向量
只有一个数为1，其他为0

# 信息论知识
## 熵
在信息论中，熵用来衡量一个随机事件的不确定性
* 熵越高，则随机变量的信息越多
* 熵越低，则随机变量的信息越少

**自信息**：一个随机事件所包含的信息量

对于一个随机变量X，当X=x时的自信息I(x)定义为
$$I(x)=-\log p(x)$$

这个公式非常符合我们的直觉，即一件事发生的概率越低，则它包含的信息量越大

### **熵：随机变量X的自信息的数学期望**

对于分布律p(x)，系统的熵为
$$H(x)=E_X[I(X)]=E_X[-\log p(x)]=-\sum_{x\in X}p(x)\log p(x)$$

性质：概率分布越均衡，熵越大
### 熵编码
在对分布p(y)的符号进行编码时，熵H(p)也是理论上最优的平均编码长度，这种编码方式称为熵编码

## 交叉熵
交叉熵是按照概率分布q的最优编码对真实分布为p的信息进行编码的长度
$$H(p,q)=E_p[-\log q(x)]=-\sum_x p(x)\log q(x)$$

在给定q的情况下，如果p和q越接近，交叉熵越小

如果p和q越远，交叉熵就越大

## KL散度
KL散度使用概率分布q来近似p时所造成的的信息损失量

KL散度是按照概率分布q的最优编码对真实分布为p的信息进行编码，其平均编码长度（即交叉熵）H(p,q)和p的最优平均编码长度（即熵）H(p)之间的差异
$$KL(p,q)=H(p,q)-H(p)=\sum_x p(x)\log\frac{p(x)}{q(x)}$$

### 应用到机器学习
以分类为例
* 真实分布$p_r(y|x)$
* 预测分布$p_\theta(y|x)$

$$KL(p_r(y|x),p_\theta (y|x))=\sum_y p_r(y|x)\log\frac{p_r(y|x)}{p_\theta (y|x)}$$
因为$p_r(y|x)$是确定的，所以以上式子正比于
$$-\sum_y p_r(y|x)\log p_\theta(y|x)$$
$$=-\log p_\theta(y^*|x)$$

# python函数
## enumerate
enumerate是python的内建函数，它可遍历每个元素（如列表、元组或字符串），组合为：索引 元素，常在for循环中使用
`enumerate(列表名)`

In [16]:
seq=['one','two','three']
for i,element in enumerate(seq):
    print(i,element)

0 one
1 two
2 three


# 常用numpy函数
## expand_dims
在指定位置插入新的轴来扩展数组形状

例如原本为一维的2个数据，axis=0，则shape变为(1,2),axis=1则shape变为(2,1)
再例如 原本为 (2,3),axis=0，则shape变为(1,2,3),axis=1则shape变为(2,1,3)

In [3]:
# 测试expand_dims函数
x = np.array([1,2,3])
print(x)
print("x.shape: ",x.shape)
y = np.expand_dims(x,axis=0)
print(y)
print("y.shape: ",y.shape)
print("y[0][1]: ",y[0][1])
y = np.expand_dims(x,axis=1)
print(y)
print("y.shape: ",y.shape)
print("y[1][0]: ",y[1][0])

[1 2 3]
x.shape:  (3,)
[[1 2 3]]
y.shape:  (1, 3)
y[0][1]:  2
[[1]
 [2]
 [3]]
y.shape:  (3, 1)
y[1][0]:  2


## concatenate
根据指定的维度，对一个元组、列表中的list或者ndarray进行连接


In [8]:
# 例子，一个2*2的数组和一个1*2的数组，在第0维进行拼接，得到一个3*2的数组
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
c = np.concatenate((a, b), axis=0)
print(c)

# 一个2*2的数组和一个2*1的数组，在第1维进行拼接，得到一个2*3的数组：
c = np.concatenate((a, b.T), axis=1)
print(c)

[[1 2]
 [3 4]
 [5 6]]
[[1 2 5]
 [3 4 6]]


上面两个简单的例子中，拼接的维度的长度是不同的，但是其他维度的长度必须是相同的，这也是使用concatenate()函数的一个基本原则，违背此规则就会报错，例如一个2\*2的数组和一个1\*2的数组，在第1维进行拼接：

In [ ]:
# 以下代码会报错
np.concatenate((a, b), axis=1)

## stack
将数组的数据按照指定的维度进行堆叠

In [10]:
a = np.array([1,2,3])
b = np.array([2,3,4])
# 按行堆叠
print(np.stack([a,b],axis=0))
# 按列堆叠
print(np.stack([a,b],axis=1))

[[1 2 3]
 [2 3 4]]
[[1 2]
 [2 3]
 [3 4]]


可以看到，进行stack的两个数组必须有相同的形状，同时，输出的结果的维度是比输入的数组都要多一维的。我们拿第一个例子来举例，两个含3个数的一维数组在第0维进行堆叠，其过程等价于先给两个数组增加一个第0维，变为1\*3的数组，再在第0维进行concatenate()操作；第二个例子则是先将两个一维数组变为3*1的二维数组，再在第1维进行concatenate()操作

## vstack
等同于`stack(arrays,axis=0)`
## hstack
等同于`stack(arrays,axis=1)`

## linalg.pinv
求矩阵（通常是非方阵）的伪逆矩阵，X的伪逆矩阵即$(X^TX)^{-1}X^T$（易知$(X^TX)^{-1}X^TX=I$）

# tensorflow
## 创建一个张量
```python
ts.constant(张量内容，dtype=数据类型（可选）)
```

In [4]:
a=tf.constant([1,5],dtype=tf.int64)
print(a)
print(a.dtype)
print(a.shape)

tf.Tensor([1 5], shape=(2,), dtype=int64)
<dtype: 'int64'>
(2,)


shape中看,隔开了几个数字这个张量就是几维的，隔开的数字为几则该维有几个元素

## 将numpy的数据类型转换为Tensor数据类型
```python
tf.convert_to_tensor(数据名,dtype=数据类型（可选）)
```

## 创建全为0的张量
`tf.zeros(维度)`
## 创建全为1的张量
`tf.ones(维度)`
## 创建全为指定值的张量
`tf.fill(维度,指定值)`

维度：
* 一维直接写个数
* 二维用[行,列]
* 多维用[n,m,j,k,...]

In [5]:
a=tf.zeros([2,3])
b=tf.ones(4)
c=tf.fill([2,2],9)
print(a)
print(b)
print(c)

tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]], shape=(2, 3), dtype=float32)
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor(
[[9 9]
 [9 9]], shape=(2, 2), dtype=int32)


## 生成正态分布的随机数，默认均值为0，标准差为1
`tf.random.normal(维度,mean=均值,stddev=标准差)`
## 生成截断式正态分布的随机数
`tf.random.truncated_normal(维度,mean=均值,stddev=标准差)`

在truncated_normal中如果随机生成数据的取值在$(\mu-2\sigma,\mu+2\sigma)$之外，则重新进行生成，保证了生成值在均值附近

## 生成均匀分布随机数 [minval,maxval)
`tf.random.uniform(维度,minval=最小值,maxval=最大值)

生成的值在[minval, maxval) 范围内遵循均匀分布

In [7]:
f=tf.random.uniform([2,2],minval=0,maxval=1)
print(f)

tf.Tensor(
[[0.4699608  0.857422  ]
 [0.07592225 0.8647555 ]], shape=(2, 2), dtype=float32)


## 强制tensor转换为该数据类型
`tf.cast(张量名,dtype=数据类型)`
## 计算张量维度上元素的最小值
`tf.reduce_min(张量名)`
## 计算张量维度上元素的最大值
`tf.reduce_max(张量名)`

In [8]:
x1=tf.constant([1,2,3],dtype=tf.float64)
print(x1)
x2=tf.cast(x1,tf.int32)
print(x2)
print(tf.reduce_min(x2),tf.reduce_max(x2))

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float64)
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(3, shape=(), dtype=int32)


在一个二维张量或数组中，可以通过调整axis等于0或1控制执行维度
* axis=0代表跨行（经度，down），axis=1代表跨列（纬度，across）
* 如果不指定axis，则所有元素参与计算



## 计算张量沿着指定纬度的平均值
`tf.reduce_mean(张量名,axis=操作轴)`
计算张量的各个维度上的元素的平均值

## 计算张量沿着指定纬度的和
`tf.reduce_sum(张量名,axis=操作轴)`

In [11]:
x = tf.constant([[1,2,3], [2,2,3]])
print(x)
print(tf.reduce_mean(x))
print(tf.reduce_mean(x, axis=1)) 
print(tf.reduce_sum(x, axis=0)) 

tf.Tensor(
[[1 2 3]
 [2 2 3]], shape=(2, 3), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor([2 2], shape=(2,), dtype=int32)
tf.Tensor([3 4 6], shape=(3,), dtype=int32)


## tf.Variable
ta.Variable()将变量标记为**“可训练”**，被标记的变量会在反向传播中记录梯度信息。神经网络训练中，常用该函数标记待训练参数

`w=tf.Variable(tf.random.normal([2,2],mean=0,stddev=1))`

## 数学运算
* 对应元素的四则运算：tf.add, tf.substract, tf.multiply, tf.divide
* 平方、次方与开方：tf.square, tf.pow, tf.sqrt
* 矩阵乘：tf.matmul

**只有维度相同的张量才可以做加减乘除的四则运算**

## 矩阵乘tf.matmul
`tf.matmul(矩阵1,,矩阵2)`

In [12]:
a=tf.ones([3,2])
b=tf.fill([2,3],3.)
print(tf.matmul(a,b))

tf.Tensor(
[[6. 6. 6.]
 [6. 6. 6.]
 [6. 6. 6.]], shape=(3, 3), dtype=float32)


## tf.data.Dataset.from_tensor_slices
切分传入张量的第一维度，生成输入特征/标签对，构建数据集
`data=tf.data.Dataset.from_tensor_slices((输入特征,标签))`

Numpy和Tensor格式都可用该语句读入数据

In [13]:
features=tf.constant([12,23,10,17])
labels=tf.constant([0,1,1,0])
dataset=tf.data.Dataset.from_tensor_slices((features,labels))
print(dataset)
for element in dataset:
    print(element)

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>
(<tf.Tensor: shape=(), dtype=int32, numpy=12>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=int32, numpy=23>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=int32, numpy=17>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


## tf.GradientTape
with结构记录计算过程，gradient求出张量的梯度
```python
with tf.GradientTape() as tape:
    若干个计算过程
grad=tape.gradient(函数,对谁求导)
```

In [15]:
with tf.GradientTape() as tape:
    w=tf.Variable(tf.constant(3.0))
    loss=tf.pow(w,2)
grad=tape.gradient(loss,w)
print(grad)

tf.Tensor(6.0, shape=(), dtype=float32)


## tf.one_hot
独热编码（one-hot encoding）：在分类问题中，常用独热码做标签，标记类别

1表示是，0表示非

标签定义：0狗尾草鸢尾  1杂色鸢尾  2弗吉尼亚鸢尾
当前标签：1
独热码：（0,1,0）

tf.onehot()函数将待转换数据，转换为one-hot形式的数据输出
`tf.one_hot(待转换数据,depth=几分类)`

In [17]:
classes=3
labels=tf.constant([1,0,2]) #输入的元素值最小为0，最大为2
output=tf.one_hot(labels,depth=classes)
print(output)

tf.Tensor(
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]], shape=(3, 3), dtype=float32)


## tf.nn.softmax
使用softmax函数将输出转换为符合概率分布的形式

当n分类的n个输出$(y_0,y_1,...,y_{n-1})$通过softmax()函数，便符合概率分布了
$$\forall x\ P(X=x)\in [0,1] 且\sum_x P(X=x)=1$$

In [18]:
y=tf.constant([1.01,2.01,-0.66])
y_pro=tf.nn.softmax(y)
print(y_pro)

tf.Tensor([0.25598174 0.69583046 0.04818781], shape=(3,), dtype=float32)


## assign_sub
用于参数的自更新

赋值操作，更新参数的值并返回

调用assign_sub前，先用tf.Variable定义变量w为可训练（可自更新）
`w.assign_sub(w要自减的内容)`

In [25]:
w=tf.Variable(4)
w.assign_sub(1)
print(w)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>


## tf.argmax
返回张量沿指定维度最大值的索引

`tf.argmax(张量名,axis=操作轴)`

In [26]:
test=np.array([[1,2,3],[2,3,4],[5,4,3],[8,7,2]])
print(test)
print(tf.argmax(test,axis=0)) # 返回每一列（经度）最大值的索引
print(tf.argmax(test,axis=1)) # 返回每一行（纬度）最大值的索引

[[1 2 3]
 [2 3 4]
 [5 4 3]
 [8 7 2]]
tf.Tensor([3 3 1], shape=(3,), dtype=int64)
tf.Tensor([2 2 0 0], shape=(4,), dtype=int64)


# 矩阵微积分
## 标量关于向量的偏导数
$$\frac{\partial y}{\partial \boldsymbol{x}}=[\frac{\partial y}{\partial x_1},\dots,\frac{\partial y}{\partial x_M}]^T$$

## 向量关于向量的偏导数
$$\frac{\partial f(\boldsymbol{x})}{\partial \boldsymbol{x}}=\begin{bmatrix} \frac{\partial y_1}{\partial x_1} & \dots  & \frac{\partial y_N}{\partial x_1} \\
\vdots & \vdots & \vdots \\
 \frac{\partial y_1}{\partial x_M} & \dots & \frac{\partial y_N}{\partial x_M}\end{bmatrix}$$

## 向量函数及其导数
$$\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}} = \boldsymbol{I}（单位矩阵）$$
$$\frac{\partial \boldsymbol{Ax}}{\partial \boldsymbol{x}}=\boldsymbol{A}^T $$
$$\frac{\partial \boldsymbol{x^TA}}{\partial \boldsymbol{x}}=\boldsymbol{A} \\$$

# 习题解答
## 2.1 分析为什么平方损失函数不适用于分类问题
分类问题中的标签，是没有连续的概念的。每个标签之间的距离也是没有实际意义的，所以预测值和标签两个向量之间的平方差这个值不能反应分类这个问题的优化程度。

比如分类 1,2,3, 真实分类是1, 而被分类到2和3错误程度应该是一样的, 但是平方损失函数的损失却不相同
## 2.2
![](https://user-images.githubusercontent.com/71121922/93963870-d6422b00-fd90-11ea-9bcb-cc83874dcd33.png)

$r^{(n)}$的作用：对每个样本的重视程度不一样
## 2.3
定理：
1. 两个矩阵的积的秩小于等于其中任何一个矩阵的秩
2. 矩阵的秩小于等于其行数和列数中更小的那个

X是$(D+1)\times N$的矩阵，因为$N<D+1$，所以$r(X)<=N$，而$r(X^T)=r(X)$，所以$r(XX^T)<=r(X)=N$

## 2.4
![](https://user-images.githubusercontent.com/15049049/83342556-817f2e00-a323-11ea-89ce-88123ae07518.png)

## 2.5
![](https://user-images.githubusercontent.com/15049049/83342561-9360d100-a323-11ea-9f4b-b289d40624fe.png)



## 2.6
### 1)
似然函数
$$L(\mu)=\frac{1}{\sqrt{2\pi}\sigma}^N \exp(-\sum_{i=1}^N \frac{(x^{(i)}-\mu)^2}{2\sigma^2}) $$
$$\frac{d\log L(\mu)}{d\mu}=0 $$
$$\frac{d(N\log \frac{1}{\sqrt{2\pi}\sigma}-\sum^N_{i=1}\frac{(x^{(i)}-\mu)^2}{2\sigma^2})}{d\mu}=0 $$
$$\sum^N_{i=1}\frac{x^{(i)}-\mu}{\sigma^2}=0 $$
$$\sum^N_{i=1}(x^{(i)}-\mu)=0 $$
$$\sum^N_{i=1}x^{(i)}=N\mu $$
$$\mu=\frac{\sum^N_{i=1}x^{(i)}}{N} $$
### 2)
![](https://user-images.githubusercontent.com/15049049/83342853-9f01c700-a326-11ea-9396-5c27b0c87031.png)

$$P(X|\mu)P(\mu)=\frac{1}{\sqrt{2\pi}\sigma}\exp(-\frac{(X-\mu)^2}{2\sigma^2})\frac{1}{\sqrt{2\pi}\sigma_0}\exp(-\frac{(\mu-\mu_0)^2}{2\sigma_0^2})$$
$$L(\mu)=\prod_{i=1}^{N}P(x^{(i)}|\mu)P(\mu)=(\frac{1}{2\pi\sigma\sigma_0})^N \exp(-\sum_{i=1}^N\frac{(x^{(i)}-\mu)^2}{2\sigma^2}-n\frac{(\mu-\mu_0)^2}{2\sigma_0^2})$$

$$\frac{d\log L(\mu)}{d\mu}=0$$
$$\frac{\sum^N_{i=1}(x^{(i)}-\mu)}{\sigma^2}=\frac{n(\mu-\mu_0)}{\sigma_0^2}$$
$$\frac{\sum^N_{i=1}x^{(i)}}{\sigma^2}+\frac{n\mu_0}{\sigma_0^2}=\frac{n\mu}{\sigma^2}+\frac{n\mu}{\sigma_0^2}$$
$$\mu=\frac{\sigma_0^2\sum^N_{i=1}x^{(i)}+n\sigma^2\mu_0}{n(\sigma^2+\sigma_0^2)}$$

## 2.7
根据大数定律，当N趋于无穷大时，似然函数中的正则项$n\frac{(\mu-\mu_0)^2}{2\sigma_0^2}$等于0